In [1]:
import tqdm
import numpy as np

# API Setup

In [2]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

True

# Dataset

In [3]:
from datasets import load_dataset
data = load_dataset("EleutherAI/wikitext_document_level", "wikitext-103-raw-v1", trust_remote_code=False)

Repo card metadata block was not found. Setting CardData to empty.


In [4]:
sample_size = 100
data_index = 38
np.random.seed(42)
data = data["train"][:sample_size]

# Preprocessing

In [5]:
# Data cleaning
import re
pattern = re.compile(r" @(.)@ ")

# Run this across the entire dataset
for i, page in enumerate(data["page"]):
    data["page"][i] = re.sub(pattern, r"\1", page)

In [6]:
# Data enrichment
def extract_metadata(data):
    title_pattern = re.compile(r"\s=\s([^=]{1,50})\s=\s")
    title = [item for item in re.findall(title_pattern, data)]
    # The regex above isn't perfect so we take the first match as the title 
    if len(title) > 0:
        title = title[0]
    else:
        title = "Unknown Title"
    return {"title": title}

In [7]:
# Load documents
from llama_index.core import Document

documents = []
for i in tqdm.tqdm(range(len(data["page"]))):
    documents.append(
        Document(
            text=data["page"][i],
            metadata=extract_metadata(data["page"][i]),
        )
    )

100%|██████████| 100/100 [00:00<00:00, 3642.69it/s]


# Chunking

We first demonstrate basic fixed size chunking with overlaps via llamaindex's [TokenTextSplitter](https://docs.llamaindex.ai/en/stable/api_reference/node_parsers/token_text_splitter/).

However, for the rest of the demo, we use a mix of two chunking methods - fixed-size chunking and content-aware chunking. Specifically, we use llamaindex's [SentenceSplitter](https://docs.llamaindex.ai/en/stable/api_reference/node_parsers/sentence_splitter/) which attempts to keep sentences and paragraphs together while maintaining chunks of roughly equal size.

We use a chunk size of 512 which is the maximum size that our embedding model (chosen further down) accepts. Within these chunks, we set an overlap of 20 tokens which corresponds to roughly 1-2 sentences.

You can examine other chunking methods implemented in llamaindex [here](https://docs.llamaindex.ai/en/stable/api_reference/node_parsers/).

### Fixed Size Chunking

In [8]:
from llama_index.core.node_parser import TokenTextSplitter
chunker = TokenTextSplitter(chunk_size=512, chunk_overlap=20)

In [9]:
sample = data["page"][data_index]
chunked = chunker.split_text(sample)
print(f"Document has ~{len(sample.split(' '))} words and was split into {len(chunked)} chunks of maximum 512 tokens.")

Document has ~9872 words and was split into 24 chunks of maximum 512 tokens.


In [10]:
# Can result in sentences being cut off in the middle
print(chunked[0])

= Michael Jordan = 
 
 Michael Jeffrey Jordan ( born February 17 , 1963 ) , also known by his initials , MJ , is an American retired professional basketball player . He is also a businessman , and principal owner and chairman of the Charlotte Hornets . Jordan played 15 seasons in the National Basketball Association ( NBA ) for the Chicago Bulls and Washington Wizards . His biography on the NBA website states : " By acclamation , Michael Jordan is the greatest basketball player of all time . " Jordan was one of the most effectively marketed athletes of his generation and was considered instrumental in popularizing the NBA around the world in the 1980s and 1990s . 
 Jordan played three seasons for coach Dean Smith at the University of North Carolina . He was a member of the Tar Heels ' national championship team in 1982 . Jordan joined the NBA 's Chicago Bulls in 1984 as the third overall draft pick . He quickly emerged as a league star , entertaining crowds with his prolific scoring . H

In [11]:
print(chunked[1])

the NBA records for highest career regular season scoring average ( 30.12 points per game ) and highest career playoff scoring average ( 33.45 points per game ) . In 1999 , he was named the greatest North American athlete of the 20th century by ESPN , and was second to Babe Ruth on the Associated Press 's list of athletes of the century . Jordan is a two-time inductee into the Basketball Hall of Fame , having been enshrined in 2009 for his individual career , and again in 2010 as part of the group induction of the 1992 United States men 's Olympic basketball team ( " The Dream Team " ) . He became a member of the FIBA Hall of Fame in 2015 . 
 Jordan is also known for his product endorsements . He fueled the success of Nike 's Air Jordan sneakers , which were introduced in 1985 and remain popular today . Jordan also starred in the 1996 feature film Space Jam as himself . In 2006 , he became part-owner and head of basketball operations for the then-Charlotte Bobcats , buying a controllin

### Content Aware Chunking

In [12]:
from llama_index.core.node_parser import SentenceSplitter
chunker = SentenceSplitter(chunk_size=512, chunk_overlap=20)

In [13]:
sample = data["page"][data_index]
chunked = chunker.split_text(sample)
print(f"Document has {len(sample.split(' '))} words and was split into {len(chunked)} chunks of maximum 512 tokens.")

Document has 9872 words and was split into 24 chunks of maximum 512 tokens.


In [14]:
# Does not cut sentences off in the middle
print(chunked[0])

= Michael Jordan = 
 
 Michael Jeffrey Jordan ( born February 17 , 1963 ) , also known by his initials , MJ , is an American retired professional basketball player . He is also a businessman , and principal owner and chairman of the Charlotte Hornets . Jordan played 15 seasons in the National Basketball Association ( NBA ) for the Chicago Bulls and Washington Wizards . His biography on the NBA website states : " By acclamation , Michael Jordan is the greatest basketball player of all time . " Jordan was one of the most effectively marketed athletes of his generation and was considered instrumental in popularizing the NBA around the world in the 1980s and 1990s . 
 Jordan played three seasons for coach Dean Smith at the University of North Carolina . He was a member of the Tar Heels ' national championship team in 1982 . Jordan joined the NBA 's Chicago Bulls in 1984 as the third overall draft pick . He quickly emerged as a league star , entertaining crowds with his prolific scoring . H

In [15]:
print(chunked[1])

Among his numerous accomplishments , Jordan holds the NBA records for highest career regular season scoring average ( 30.12 points per game ) and highest career playoff scoring average ( 33.45 points per game ) . In 1999 , he was named the greatest North American athlete of the 20th century by ESPN , and was second to Babe Ruth on the Associated Press 's list of athletes of the century . Jordan is a two-time inductee into the Basketball Hall of Fame , having been enshrined in 2009 for his individual career , and again in 2010 as part of the group induction of the 1992 United States men 's Olympic basketball team ( " The Dream Team " ) . He became a member of the FIBA Hall of Fame in 2015 . 
 Jordan is also known for his product endorsements . He fueled the success of Nike 's Air Jordan sneakers , which were introduced in 1985 and remain popular today . Jordan also starred in the 1996 feature film Space Jam as himself . In 2006 , he became part-owner and head of basketball operations fo

### Perform Chunking on Dataset

Now we perform chunking on our entire dataset. We use the chunker's in-built function to do this on our list of documents. We get a list of [TextNodes](https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_nodes/) as output. 

In [16]:
nodes = chunker.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/100 [00:00<?, ?it/s]

In [17]:
print(f"Documents before chunking: {len(documents)}")
print(f"Documents after chunking: {len(nodes)}")
nodes[0]

Documents before chunking: 100
Documents after chunking: 837


TextNode(id_='def5af20-e66e-4ec5-906c-ec6ca3c723cf', embedding=None, metadata={'title': 'Valkyria Chronicles III'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3e5f57ab-4225-4edb-8a71-4cb4706ad4f1', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'title': 'Valkyria Chronicles III'}, hash='f7aadfb478d20e04be770cd882b5e6a44c185eb28a53810838586313c39ccc7c'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='1fb81062-ebd0-4d4c-9ca6-eec39b25ac16', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='32c32274359d6ec7e58a31e940b4b433c53354c2fb60611c7cc6bd2c324d075c')}, text='= Valkyria Chronicles III = \n \n Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role-playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in Jan